In [ ]:
df = pd.read_csv('/kaggle/input/supply-chain-optimization-for-a-fmcg-company/FMCG_data.csv')
# X = df.iloc[:, :-1]
#  y = df.iloc[:, -1]

In [ ]:
df.head()

# Looking at the basic schema of the dataframe

In [ ]:
df.info()

# Showing some features with missing values

In [ ]:
df.loc[:,['wh_est_year','approved_wh_govt_certificate']].head(20)

# How different features are correlated with each other 

In [ ]:
df.corr()

# Dropping the rows having null values in wh_est_year

In [ ]:
df.dropna(subset=['wh_est_year'],inplace=True)

# Checking for left out features with missing values 

In [ ]:
df.info()

# Analysing the null values 

In [ ]:
df.loc[:,['workers_num','approved_wh_govt_certificate']].head(60)

# Dropping the rows with this feature having null values 

In [ ]:
df.dropna(subset=['approved_wh_govt_certificate'],inplace=True)

# Looking for more such features 

In [ ]:
df.info()

# Locating null values for this feature.

In [ ]:
df.loc[:,['workers_num']].iloc[60:,:].head(60)

# Filling missing values with mean using sklearn 

In [ ]:
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(missing_values=np.nan,strategy='mean')
imputer.fit(df.loc[:,['workers_num']])
df.loc[:,['workers_num']]=imputer.transform(df.loc[:,['workers_num']])

# Showing the results 

In [ ]:
df.loc[:,['workers_num']].iloc[60:,:].head(60)

# Ensuring the data is clean 

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

# Dropping the unnecessary features 

In [ ]:
df.drop(['Ware_house_ID','WH_Manager_ID'],axis=1,inplace=True)

In [ ]:
l=['num_refill_req_l3m','transport_issue_l1y','Competitor_in_mkt','retail_shop_num','distributor_num','flood_impacted','flood_proof','electric_supply','dist_from_hub','workers_num','wh_est_year','storage_issue_reported_l3m','temp_reg_mach','wh_breakdown_l3m','govt_check_l3m']

In [ ]:
X=df.loc[:,l]
y=df.iloc[:,-1]

# Performing train test split

In [ ]:
from sklearn.model_selection import train_test_split
X_train0, X_test0, y_train0, y_test0 = train_test_split(X, y, test_size = 0.2, random_state = 1)

# Displaying basic MAPE of model trained on non-categorical features.  

In [ ]:
#Training the model
from sklearn.linear_model import LinearRegression
model=LinearRegression()
model.fit(X_train0,y_train0)
# Predicting on test data 
y_pred=model.predict(X_test0)
from sklearn.metrics import mean_absolute_percentage_error
# Baseline mape for linear regression
basic_mape=mean_absolute_percentage_error(y_pred,y_test0)
print(basic_mape)

# One hot Encoding the categorical fetaures of the entire dataframe 

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
transformed_df=df
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0, 1, 2, 3,8,18])], remainder='passthrough')
transformed = ct.fit_transform(df)
transformed_columns = ct.get_feature_names_out()
transformed_df = pd.DataFrame(transformed, columns=transformed_columns, index=df.index)


# Visualizing the correlation of the transformed dataframe. 

In [ ]:
transformed_df
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(25,12))
sns.heatmap(data=transformed_df.corr())

In [ ]:
transformed_df.info()

# Creating dependent and independent variables

In [ ]:
X_transformed_df= transformed_df.iloc[:,:-1]
y = df.iloc[:, -1]

# Performing train test splt

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_transformed_df, y, test_size = 0.2, random_state = 1)

# Looking for the change after split

In [ ]:
X.info()

In [ ]:
X_train.info()

**Printing basic mape for comparison**

In [ ]:
print(basic_mape)

# Obtaining new MAPE

In [ ]:
from sklearn.linear_model import LinearRegression
model=LinearRegression()
model.fit(X_train,y_train)
# Predicting on test data 
y_pred=model.predict(X_test)
from sklearn.metrics import mean_absolute_percentage_error
# Baseline mape for linear regression
mean_absolute_percentage_error(y_pred,y_test)

**Clearly there is 11.1% reduction in MAPE after one hot encoding.** 

# converting numpy array y_pred to series for results visualisation.

In [ ]:
s1=pd.Series(data=y_pred,index=y_test.index)

# Plotting predictions and ground truth values

In [ ]:
sns.scatterplot(x=s1,y=y_test)

# Plotting noise and ground truth values

In [ ]:
s2=y_test-y_pred
sns.scatterplot(data=s2)

In [ ]:
sns.histplot(data=s2)

# Noise has some correlation with ground truth values

In [ ]:
s2.corr(y_test)

# Looking for features correlating with noise.

In [ ]:
X_test.corrwith(s2)

# Initial shape of test data 

In [ ]:
X_test.shape

In [ ]:
u=int(X_test.shape[0]/2)
print(u)

**Splitting the test data into valid and test1(new test) each having equal amount of data.**

In [ ]:
X_val=X_test.iloc[:u,:]
X_test1=X_test.iloc[u:,:]
y_val=y_test[:u]
y_test1=y_test[u:]

**Training RandomForestRegressor model on X_valid and predictions (generated by linear regression) corresponding to X_val.**  

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
regressor.fit(X_val, y_pred[:u])
# Predicting on new test data 
y_pred1=regressor.predict(X_test1)
mean_absolute_percentage_error(y_pred1,y_test1)

**Training RandomForestRegressor model on X_valid and noise (generated by linear regression) corresponding to X_val.**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor1 = RandomForestRegressor(n_estimators = 10, random_state = 0)
regressor1.fit(X_val, y_val-y_pred[:u])
# Predicting on new test data 
y_pred1=regressor1.predict(X_test1)
mean_absolute_percentage_error(y_pred1+y_pred[u:],y_test1)

**Clearly 24.22% decrease in MAPE is visible after using noise instead of predictions.**

# **Thank You**